In [1]:
import importlib
#import sutils; importlib.reload(sutils)
from sutils import *

import keras
import gensim
import re
import pickle
import keras.backend as K

from keras import initializers
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, TensorBoard
from keras.callbacks import LambdaCallback


from recurrentshop import *
import seq2seq
from seq2seq.models import AttentionSeq2Seq,SimpleSeq2Seq, Seq2Seq

import tensorflow as tf
from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


In [2]:
print(keras.__version__)
print(tf.__version__)

2.0.8
1.3.0


In [3]:
from gensim.models import word2vec

path = '../neural_translation_en_de/'
dpath = '../neural_translation_en_de/translate/'

In [4]:

# ### Set up Regex and tokenize for use later

re_mult_space = re.compile(r"  *")
re_mw_punc = re.compile(r"(\w[’'])(\w)")
re_punc = re.compile("([\"().,;:/_?!—])")
re_apos = re.compile(r"(\w)'s\b")


def simple_toks(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' ')
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()

In [5]:

# ## Load the PreProcessed data
# 
# Here we load all the data 

data = load(dpath+'nmt_datawmtsmall_sos_eos_unk.pkl')
look_ups = load(dpath+'look_upswmtsmall_sos_eos_unk.pkl')
fr_train = data['fr_train']
fr_test = data['fr_test']
en_train = data['en_train']
en_test = data['en_test']
en_w2id = look_ups['en_w2id']
fr_vocab = look_ups['fr_vocab']
en_vocab = look_ups['en_vocab']
en_embs = look_ups['en_embs']
fr_embs = look_ups['fr_embs']

questions = load(dpath+'questionswmt.pkl')
#print(questions[10])
en_qs, fr_qs = zip(*questions)

In [6]:
print(fr_train.shape)
print(en_train.shape)

(100000, 100)
(100000, 100)


In [7]:

# ## Model

# #### Create some Keras Callbacks to handle early stopping and Learning Rate changes

# In[10]:


reduce_LR = ReduceLROnPlateau(monitor='val_loss',factor = 0.5, patience=0,cooldown=1, min_lr = 0.00001)
early_stopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=4,verbose=0,mode='auto')

import math

# learning rate schedule for dropping every 10 epochs
def LRDropping(epoch):
    initial_lrate = 0.001
    drop = 0.9
    epochs_drop = 3.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

# try at manual setting of LR for Epochs
def fixed_dropping(epoch):
    if epoch < 2: 
        lrate = 0.01
    elif epoch < 4: 
        lrate = 0.001
    elif epoch < 7: 
        lrate = 0.0005
    else:
        lrate = 0.0001
    print(lrate)
    return lrate

LRDrop = LearningRateScheduler(fixed_dropping)


# In[11]:

tbCallBack = TensorBoard(log_dir='/data/model_newGraphadvsmall_sos_eos_unk', write_graph=True)
modelCallback = ModelCheckpoint(
            'model_checkpoint_advsmall_sos_eos_unk.{epoch:03d}-{loss:.3f}.hdf5',
            monitor='val_loss', verbose=1, save_best_only=False,
            save_weights_only=True, mode='auto',
            period=1)

# creating different sets of Params to easily import into the model at train time
params = {'verbose': 1, 'callbacks': [TQDMNotebookCallback(),reduce_LR,early_stopping,tbCallBack,modelCallback]}
params2 = {'verbose': 1, 'callbacks': [LRDrop,TQDMNotebookCallback(),reduce_LR,early_stopping]}
params3 = {'verbose': 1, 'callbacks': [LRDrop,TQDMNotebookCallback(),reduce_LR,early_stopping]}

In [8]:


# #### Set some parameters for the model

# In[12]:


lr = 1e-3
maxlen = 100
dim_en_vec = 200
n_en_vec = 400000
dim_fr_vec = 200

vocab_size = len(fr_vocab) #the output vocab # embeddings.shape[0]
embedding_size = 200 #The english inputs embeddings embeddings.shape[1]


fr_wgts = [fr_embs.T, np.zeros((len(fr_vocab,)))]

In [ ]:
# ### The model itself

# Base Model big
inp = Input((maxlen,))
x = Embedding(40003, dim_en_vec, input_length=maxlen,
              weights=[en_embs], trainable=False)(inp)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = LSTM(128, return_sequences=True)(x)
x = TimeDistributed(Dense(dim_fr_vec))(x)
x = TimeDistributed(Dense(40003, weights=fr_wgts))(x)
x = Activation('softmax')(x)

model = Model(inp, x)
model.compile('adam', 'sparse_categorical_crossentropy')


K.set_value(model.optimizer.lr, lr)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 200)          8000600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 256)          336896    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 256)          394240    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 128)          197120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 200)          25800     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 40003)        8040603   
__________

In [ ]:
hist=model.fit(en_train, np.expand_dims(fr_train,axis=-1), batch_size=96, epochs=10, **params, 
               validation_data=[en_test, np.expand_dims(fr_test,axis=-1)])

Train on 100000 samples, validate on 10000 samples


HBox(children=(IntProgress(value=0, description='Training', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=100000), HTML(value='')))

Epoch 1/10
100000/100000 [==============================] - 1191s - loss: 1.9079 - val_loss: 1.7004


HBox(children=(IntProgress(value=0, description='Epoch 1', max=100000), HTML(value='')))

Epoch 2/10
100000/100000 [==============================] - 1185s - loss: 1.6952 - val_loss: 1.6601


HBox(children=(IntProgress(value=0, description='Epoch 2', max=100000), HTML(value='')))

Epoch 3/10
100000/100000 [==============================] - 1186s - loss: 1.6486 - val_loss: 1.6160


HBox(children=(IntProgress(value=0, description='Epoch 3', max=100000), HTML(value='')))

Epoch 4/10
 14976/100000 [===>..........................] - ETA: 987s - loss: 1.6156

In [ ]:
#plot_train(hist)
for a in hist.history:
    print(a, hist.history[a])


In [ ]:
weight_identifier = "trans_testing_basic_wmt_advsmall_sos_eos_unk"
model.save_weights(dpath + weight_identifier + '.h5')
model.load_weights(dpath + weight_identifier + '.h5')

In [ ]:
# ## Testing

def sent2ids(sent):
    sent = simple_toks(sent)
    ids = []
    if SOS:
        ids.append(en_w2id["<SOS>"])
    for t in sent:
        try:
            ids.append(en_w2id[t])
        except KeyError:
            if UNK:
                ids.append(en_w2id["<UNK>"])
            else:
                pass
    if EOS:
        ids.append(en_w2id["<EOS>"])
    return pad_sequences([ids], maxlen, padding="post", truncating="post")

def en2fr(sent): 
    ids = sent2ids(sent)
    tr_ids = np.argmax(model.predict(ids), axis=-1)
    return ' '.join(fr_vocab[i] for i in tr_ids[0] if i>0)


In [ ]:
en2fr("what is the size of canada?")


# In[26]:


en2fr("what is the size of australia?")


# In[27]:


en2fr("What is light?")


# In[28]:


en2fr("Why is the Arctic ozone layer thicker than the Antarctic ozone layer?")


# In[29]:


#print(qs[9])
en2fr("Which province is the most populated?")


# In[30]:


en2fr("Who are we?")


# In[31]:


en2fr("What would we do without it?")


# In[32]:


en2fr("Hello Tom")

## Prediction

In [ ]:
import os

source_file = "C:/Users/Nicolas/CloudStation/PA/DE_EN_(tatoeba)_validation_english_only.txt"
if os.path.exists(source_file) is False:
    exit("source file does not exists")

source_sentences = open(source_file, encoding='UTF-8').read().split('\n')
print(len(source_sentences))

translated_sentences = []
i = 0
for sent in source_sentences:
    if i % int((len(source_sentences) / 100)) == 0:
        print(i)
    translated_sentences.append(en2fr(sent))
    i += 1
print(len(translated_sentences))

In [ ]:
out_file = os.path.join(os.path.abspath(os.path.join(source_file, os.pardir)), weight_identifier + ".pred")
with(open(out_file, 'w', encoding='utf8')) as file:
    for sent in translated_sentences:
        file.write(sent + '\n')